# Dataframe from a table on Wikipedia 
## _This notebook creates a Dataframe from Toronto Post Codes table from Wikipedia_
We use the BeautifulSoup library

## 1. Get a Dataframe from a table

First, [install and] import dependancies:

In [ ]:
#!pip install beautifulsoup4
#!pip install lxml
#!pip show beautifulsoup4

In [1]:
from bs4 import BeautifulSoup
import requests
import lxml
import html5lib
import pandas as pd

The site we want to parse is [here](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

In [2]:
wikipedia_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
# Getting webpage to 'requests' object
raw_page = requests.get(wikipedia_link)

In [4]:
# Initialization of 'BeautifulSoup' object with parser 'lxml'
# and passing the text from 'requests' object as input
soup = BeautifulSoup(raw_page.text,'lxml')

In [5]:
toronto = soup.table
#print(toronto.prettify())

The structure of the table is as follows:
  < tr >
   < td >
    Postcode
   < /td >
   < td >
    Borough
   < /td >
   < td >
    Neighbourhood
   < /td >
  < /tr >
Now we can create separate lists for each column and populate them in a loop

In [6]:
postcode = []
borough = []
neighbourhood = []

for row in toronto.find_all('tr'):
    try:
        postcode.append(row.find_all('td')[0].text)
        borough.append(row.find_all('td')[1].text)
        neighbourhood.append(row.find_all('td')[2].text)
    except:
        #ignore the 1st row tagged with <th>
        pass

In [7]:
#Check the equality of arrays' lengths

print('postcode: ',len(postcode))
print('borough: ',len(borough))
print('neighbourhood: ',len(neighbourhood))

postcode:  288
borough:  288
neighbourhood:  288


Now we can create and preview Dataframe:

In [8]:
df = pd.DataFrame({'Postcode':postcode,'Borough':borough,'Neighbourhood':neighbourhood})
df['Neighbourhood'].replace(regex='\\n',value='',inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## 2. Cleaning the table

Ignore cells with a borough that is Not assigned.

In [9]:
df = df[df['Borough'] != 'Not assigned']
df.reset_index(drop=True,inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


We have several neighborhoods for some of postcodes:

In [10]:
print(len(df['Postcode'].unique()))
print(len(df['Neighbourhood'].unique()))

103
209


Let's combine them:

In [11]:
df = df.groupby('Postcode').agg(
    {'Borough': lambda x: list(x)[0],
     'Neighbourhood': lambda x:', '.join(map(str, list(x)))}).reset_index()

In [12]:
print(len(df['Postcode'].unique()))
print(len(df['Neighbourhood'].unique()))
df.head()

103
103


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Let's find 'Not assigned' Neighborhoods and assign them the borough name:

In [13]:
df['Neighbourhood'].replace('Not assigned',value=df['Borough'],inplace=True)

In [14]:
df.shape

(103, 3)